<a href="https://colab.research.google.com/github/Gongmengjie/speaker_classification/blob/main/conformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install conformer

In [ ]:
!wget https://www.dropbox.com/s/vw324newiku0sz0/Dataset.tar.gz.aa?dl=0
!wget https://www.dropbox.com/s/z840g69e7lnkayo/Dataset.tar.gz.ab?dl=0
!wget https://www.dropbox.com/s/hl081e1ggonio81/Dataset.tar.gz.ac?dl=0
!wget https://www.dropbox.com/s/fh3zd8ow668c4th/Dataset.tar.gz.ad?dl=0
!wget https://www.dropbox.com/s/ydzygoy2pv6gw9d/Dataset.tar.gz.ae?dl=0
!cat Dataset.tar.gz.* | tar zxvf -

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from conformer import ConformerBlock

class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir, "mapping.json")
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]

    # Load metadata of training data.
    metadata_path = Path(data_dir, "metadata.json")
    metadata = json.load(open(metadata_path))["speakers"]

    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, d_model=512, n_spks=600, dropout=0.1): 
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    # d_model = input_dim，可以改的大一些
    # 前馈层
    self.prenet = nn.Linear(40, d_model)


    # 1.传统的transformer
    # self.encoder_layer = nn.TransformerEncoderLayer(
    #  d_model=d_model, dim_feedforward=256, nhead=8
    # )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=4)

    # 2.conformer(相对于单纯的transformer效果有所提升)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.conformer_block = ConformerBlock(
    dim = 512,
    dim_head = 64,
    heads = 8,
    ff_mult = 4,
    conv_expansion_factor = 2,
    conv_kernel_size = 31,
    attn_dropout = 0.,
    ff_dropout = 0.,
    conv_dropout = 0.,
  )
    
    # 前馈层
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model) = (32, 128, 512)
    out = self.prenet(mels)
    
    out = self.conformer_block(out)
    # mean pooling
    stats = out.mean(dim=1) # (batch_size, d_model)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)

In [7]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)

In [9]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())

[Info]: Use cuda now!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:57<00:00, 17.06 step/s, accuracy=0.47, loss=2.47, step=2000]
Valid: 100% 6944/6944 [00:12<00:00, 561.93 uttr/s, accuracy=0.31, loss=3.18]
Train: 100% 2000/2000 [01:47<00:00, 18.58 step/s, accuracy=0.50, loss=2.17, step=4000]
Valid: 100% 6944/6944 [00:10<00:00, 650.20 uttr/s, accuracy=0.43, loss=2.55]
Train: 100% 2000/2000 [01:47<00:00, 18.67 step/s, accuracy=0.53, loss=2.15, step=6000]
Valid: 100% 6944/6944 [00:10<00:00, 666.40 uttr/s, accuracy=0.50, loss=2.20]
Train: 100% 2000/2000 [01:47<00:00, 18.69 step/s, accuracy=0.62, loss=1.70, step=8000]
Valid: 100% 6944/6944 [00:10<00:00, 657.22 uttr/s, accuracy=0.57, loss=1.81]
Train: 100% 2000/2000 [01:47<00:00, 18.66 step/s, accuracy=0.75, loss=1.33, step=1e+4]
Valid: 100% 6944/6944 [00:10<00:00, 656.44 uttr/s, accuracy=0.61, loss=1.64]
Train:   0% 3/2000 [00:00<02:52, 11.57 step/s, accuracy=0.69, loss=0.99, step=1e+4]

Step 10000, best model saved. (accuracy=0.6070)


...

Step 50000, best model saved. (accuracy=0.8416)


Train: 100% 2000/2000 [01:47<00:00, 18.67 step/s, accuracy=0.91, loss=0.42, step=52000]
Valid: 100% 6944/6944 [00:10<00:00, 636.46 uttr/s, accuracy=0.85, loss=0.73]
Train: 100% 2000/2000 [01:47<00:00, 18.67 step/s, accuracy=0.97, loss=0.14, step=54000]
Valid: 100% 6944/6944 [00:11<00:00, 631.14 uttr/s, accuracy=0.85, loss=0.74]
Train: 100% 2000/2000 [01:47<00:00, 18.65 step/s, accuracy=0.97, loss=0.09, step=56000]
Valid: 100% 6944/6944 [00:10<00:00, 641.93 uttr/s, accuracy=0.85, loss=0.71]
Train: 100% 2000/2000 [01:47<00:00, 18.64 step/s, accuracy=0.97, loss=0.13, step=58000]
Valid: 100% 6944/6944 [00:10<00:00, 637.99 uttr/s, accuracy=0.86, loss=0.69]
Train: 100% 2000/2000 [01:47<00:00, 18.59 step/s, accuracy=0.94, loss=0.15, step=6e+4]
Valid: 100% 6944/6944 [00:11<00:00, 623.08 uttr/s, accuracy=0.86, loss=0.70]
Train:   0% 3/2000 [00:00<02:46, 12.01 step/s, accuracy=1.00, loss=0.01, step=6e+4]

Step 60000, best model saved. (accuracy=0.8600)


Train: 100% 2000/2000 [01:47<00:00, 18.62 step/s, accuracy=1.00, loss=0.04, step=62000]
Valid: 100% 6944/6944 [00:11<00:00, 625.59 uttr/s, accuracy=0.86, loss=0.68]
Train: 100% 2000/2000 [01:47<00:00, 18.60 step/s, accuracy=0.91, loss=0.29, step=64000]
Valid: 100% 6944/6944 [00:12<00:00, 575.48 uttr/s, accuracy=0.86, loss=0.66]
Train: 100% 2000/2000 [01:47<00:00, 18.62 step/s, accuracy=0.97, loss=0.09, step=66000]
Valid: 100% 6944/6944 [00:11<00:00, 606.72 uttr/s, accuracy=0.86, loss=0.68]
Train: 100% 2000/2000 [01:47<00:00, 18.68 step/s, accuracy=0.97, loss=0.19, step=68000]
Valid: 100% 6944/6944 [00:11<00:00, 600.06 uttr/s, accuracy=0.86, loss=0.67]
Train: 100% 2000/2000 [01:48<00:00, 18.41 step/s, accuracy=0.97, loss=0.10, step=7e+4]
Valid: 100% 6944/6944 [00:11<00:00, 612.43 uttr/s, accuracy=0.87, loss=0.62]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.8695)
